In [1]:
"""
Kaggle Submission Notebook for CSIRO Biomass Prediction
========================================================
This notebook runs inference using trained models and creates submission.csv

Usage:
1. Train models locally: python train.py --config configs/exp/exp001.yaml
2. Upload model weights and code to Kaggle (run cell-2 locally)
3. Upload this notebook to Kaggle with the data sources attached
4. Run this notebook on Kaggle
"""

import os
import sys

# Kaggle環境ではコードDatasetからインポートするためにsys.pathを設定
# Dataset名: csiro-biomass-codes-{EXP_NAME} (例: csiro-biomass-codes-001)
if os.getenv("KAGGLE_DATA_PROXY_TOKEN"):
    sys.path.insert(0, "/kaggle/input/csiro-biomass-codes-013")

import config
import pandas as pd

# Check environment
print(f"Running in Kaggle: {config.IS_KAGGLE_ENV}")
print(f"Input directory: {config.INPUT_DIR}")
print(f"Output directory: {config.OUTPUT_DIR}")
print(f"Artifact directory: {config.ARTIFACT_DIR}")


Running in Kaggle: False
Input directory: /workspace/data/input
Output directory: /workspace/data/output/013/1
Artifact directory: /workspace/data/output


In [ ]:
# # Run inference (DINOv3)
# from inference_dinov3 import kaggle_inference

# # Configuration - Change run_name to match your best run
# RUN_NAME = "001_exp013_dinov3__lr-0_002__weight_decay-0_01"
# FOLDS = None  # None = use all 5 folds (0-4), or specify list like [0, 1, 2]
# IMG_SIZE = 960  # Image size (must be divisible by 16)
# WEIGHT_TYPE = "best"  # "best" (lowest loss) or "last" (final epoch)

# # Run inference and create submission
# # TTA (4 flip variants) is always enabled
# submission_df = kaggle_inference(
#     run_name=RUN_NAME,
#     folds=FOLDS,
#     img_size=IMG_SIZE,
#     weight_type=WEIGHT_TYPE,
# )

# # Save submission
# submission_df.to_csv("submission.csv", index=False)
# print("Submission saved to submission.csv")
# print(f"Submission shape: {submission_df.shape}")
# submission_df.head(10)


In [2]:
# Upload artifacts to Kaggle (run this locally after training)
# Uncomment and run this cell to upload trained models and code

# ============================================================
# アップロードする experiment_dir を指定
# data/output/013/1/ 内のディレクトリを確認して指定してください
# ============================================================
EXPERIMENT_DIR = "20251227_174406_exp013_dinov3"  # ← ここを変更

if not config.IS_KAGGLE_ENV:
    import shutil

    from src.kaggle_utils.customhub import dataset_upload, model_upload

    experiment_path = config.OUTPUT_DIR / EXPERIMENT_DIR
    if not experiment_path.exists():
        raise FileNotFoundError(f"Experiment directory not found: {experiment_path}")

    print(f"Uploading experiment: {experiment_path}")

    # Copy backbone.pth to experiment directory for upload
    # backbone.pth is saved at OUTPUT_DIR/backbone.pth during feature extraction
    backbone_src = config.OUTPUT_DIR / "backbone.pth"
    backbone_dst = experiment_path / "backbone.pth"
    if backbone_src.exists() and not backbone_dst.exists():
        shutil.copy2(backbone_src, backbone_dst)
        print(f"Copied backbone.pth to {backbone_dst}")
    elif backbone_dst.exists():
        print(f"backbone.pth already exists at {backbone_dst}")
    else:
        print(f"WARNING: backbone.pth not found at {backbone_src}")

    # モデル重みを Kaggle Model としてアップロード
    # experiment_dir 内の全 run_name をアップロード
    model_upload(
        handle=config.ARTIFACTS_HANDLE,
        local_model_dir=experiment_path,
        update=False,  # Set to True to update existing model
    )

    # 実験コードを Kaggle Dataset としてアップロード
    # experiments/013/ ディレクトリのみ（プロジェクト全体ではない）
    dataset_upload(
        handle=config.CODES_HANDLE,
        local_dataset_dir=config.EXP_DIR,  # experiments/013/ のみ
        update=True,
    )


Uploading experiment: /workspace/data/output/013/1/20251227_174406_exp013_dinov3
Copied backbone.pth to /workspace/data/output/013/1/20251227_174406_exp013_dinov3/backbone.pth


Model instance not found


dst_dir=/tmp/tmpsza2clin
tree
├── 001_dinov3_head__lr-0_002__weight_decay-0_01
│   ├── config.yaml
│   ├── logs
│   │   ├── metrics.csv
│   │   └── train.log
│   ├── plots
│   │   └── training_curves.png
│   ├── predictions.csv
│   ├── submission.csv
│   └── weights
│       ├── best_fold0.pth
│       ├── best_fold1.pth
│       ├── best_fold2.pth
│       ├── best_fold3.pth
│       ├── best_fold4.pth
│       ├── last_fold0.pth
│       ├── last_fold1.pth
│       ├── last_fold2.pth
│       ├── last_fold3.pth
│       └── last_fold4.pth
├── 002_dinov3_head__lr-0_002__weight_decay-0_001
│   ├── config.yaml
│   ├── logs
│   │   └── train.log
│   ├── plots
│   └── weights
│       ├── best_fold0.pth
│       ├── best_fold1.pth
│       ├── best_fold2.pth
│       ├── best_fold3.pth
│       ├── best_fold4.pth
│       ├── last_fold0.pth
│       ├── last_fold1.pth
│       ├── last_fold2.pth
│       └── last_fold3.pth
└── backbone.pth
Starting upload for file 002_dinov3_head__lr-0_002__weight_decay-0_0

100%|██████████| 92.0M/92.0M [00:05<00:00, 18.4MB/s]


Upload successful: 002_dinov3_head__lr-0_002__weight_decay-0_001.zip (92MB)
Starting upload for file 001_dinov3_head__lr-0_002__weight_decay-0_01.zip


...

Upload successful: configs.zip (918B)
Starting upload for file train_head.py


100%|██████████| 29.1k/29.1k [00:00<00:00, 34.9kB/s]


Upload successful: train_head.py (29KB)
Starting upload for file config.py


100%|██████████| 2.88k/2.88k [00:00<00:00, 3.80kB/s]


Upload successful: config.py (3KB)
